<a href="https://colab.research.google.com/github/MukeshSK75/Deep-Learning-Practice/blob/main/TrafficLightsADL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def create_traffic_signal(label, img_size=28):
    image = np.zeros((img_size, img_size), dtype=np.float32)
    center_x = img_size // 2
    if label == 0:  # red
        image[6, center_x] = 1.0
    elif label == 1:  # yellow
        image[14, center_x] = 1.0
    else:  # green
        image[22, center_x] = 1.0
    return image

labels = np.random.randint(0, 3, 3000)
images = np.array([create_traffic_signal(label) for label in labels])
images = images.reshape(-1, 28, 28, 1)
images = (images - 0.5) * 2

x_train, _ = train_test_split(images, test_size=0.2, random_state=42)

def build_generator():
    model = models.Sequential([
        layers.Dense(128, input_shape=(100,), activation='relu'),
        layers.Dense(784, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model

def build_discriminator():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False

gan_input = layers.Input(shape=(100,))
generated_img = generator(gan_input)
gan_output = discriminator(generated_img)

gan = models.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

def show_sample_images(generator, n=5):
    noise = np.random.normal(0, 1, (n, 100))
    gen_imgs = generator.predict(noise, verbose=0)
    gen_imgs = 0.5 * gen_imgs + 0.5

    plt.figure(figsize=(10, 2))
    for i in range(n):
        plt.subplot(1, n, i + 1)
        plt.imshow(gen_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()

def train_gan(epochs=1000, batch_size=32):
    for epoch in range(epochs):

        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_imgs = x_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_imgs = generator.predict(noise, verbose=0)

        real_y = np.ones((batch_size, 1))
        fake_y = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, real_y)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)

        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_y)

        if epoch % 100 == 0:
            print(f"Epoch {epoch} | D loss: {0.5 * np.add(d_loss_real, d_loss_fake)} | G loss: {g_loss}")
            show_sample_images(generator)

train_gan(epochs=1000, batch_size=64)